# LEV4Rec RS generator
Generated recommender system from model.

In [1]:
from surprise import Dataset
import pandas as pd
from surprise import Reader



df = pd.read_csv('crossRecData.csv')
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['LibID', 'rating', 'ProjectID']], reader)



n_splits=10


from surprise.model_selection import KFold
from collections import defaultdict
kf = KFold(n_splits=n_splits)



is_user_based=False
neighborhood=40
cutoff=5
sim_funct='cosine'

sim_settings = {'name': sim_funct,
               'user_based': is_user_based  
               }
from surprise import KNNWithMeans
algo = KNNWithMeans(k=neighborhood, sim_options=sim_settings)




threshold = 3.5
k=10
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    

    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0


        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0




    precision= sum(prec for prec in precisions.values()) / len(precisions)
    recall =sum(rec for rec in recalls.values()) / len(recalls)
    
    print("average precision",precision)
    print("average recall", recall)


    f1_measure=(2*precision* recall) / (recall + precision)
    print("average f1_measure",f1_measure)



 

Computing the cosine similarity matrix...
Done computing similarity matrix.
average precision 0.9985424865180004
average recall 0.9929061463776994
average f1_measure 0.9957163402616576
Computing the cosine similarity matrix...
Done computing similarity matrix.
average precision 0.9989049496276828
average recall 0.9929098651650877
average f1_measure 0.9958983852128352
Computing the cosine similarity matrix...
Done computing similarity matrix.
average precision 0.9985695422535211
average recall 0.9921546473705337
average f1_measure 0.9953517591571501
Computing the cosine similarity matrix...
Done computing similarity matrix.
average precision 0.9985005248163142
average recall 0.9925454067726782
average f1_measure 0.9955140600655146
Computing the cosine similarity matrix...
Done computing similarity matrix.
average precision 0.9973273942093541
average recall 0.9912224015474992
average f1_measure 0.9942655264924345
Computing the cosine similarity matrix...
Done computing similarity matrix.